In [1]:
using LinearAlgebra

In [2]:
module MyFiniteDiff
using LinearAlgebra
function _e(T, i, n)
    v = zeros(T, n)
    v[i] = one(T)
    return v
end

"Find the numerical gradient of f : R^2 -> R using forward or centered differences"
function grad_finite_diff(x, f, h, type_)   
    n = size(x, 1)
    grad = similar(x)
    
    fx = f(x)
    
    e(i) = _e(eltype(x), i, n)
    
    for i in 1:n
        if type_ == :fw
            grad[i] = (f(x + h*e(i)) - fx) / h
        end
        if type_ == :c
            grad[i] = (f(x + h*e(i)) - f(x - h*e(i))) / 2h
        end
    end
    
    return grad
end

function hess_finite_diff(x, f, h)
    n = size(x, 1)
    grad = similar(x, n, n)
    
    e(i) = _e(eltype(x), i, n)
    
    for i in 1:n
        for j in 1:n
            # Just calculate the upper triangle
            if i < j
                grad[i, j] = (
                    f(x + h*e(i) + h*e(j)) - f(x + h*e(i)) - f(x + h*e(j)) + f(x)
                ) / (h^2)
            end
            if i == j
                grad[i, i] = (
                    f(x + h*e(i)) - 2*f(x) + f(x - h*e(i))
                ) / (h^2)
            end
        end
    end
    
    # And then copy the upper part into the lower part
    return Symmetric(grad)
end

function jacobian_finite_diff(F, x, h, type_)
    n = size(x, 1)
    jac = similar(x, n, n)
    
    e(i) = _e(eltype(x), i, n)
    
    Fx = F(x)
    
    
    for i in 1:n
        if type_ == :fw
            ∇ₓᵢF = (F(x + h * e(i)) - Fx) / h
            jac[:, i] = ∇ₓᵢF
        end
         if type_ == :c
            ∇ₓᵢF = (F(x + h * e(i)) - F(x - h * e(i))) / 2h
            jac[:, i] = ∇ₓᵢF
        end
    end
    
    return jac
end

function newton_method_backtrack_finite_diff(x0, f, kmax, tollgrad, alpha0, c1, rho, btmax, FDgrad, FDhess)
    gradf(x) = grad_finite_diff(x, f, sqrt(eps()), FDgrad)
    
    hessf(x) = error("Undefined hessf")
    # TODO(Andrea): finish
    if FDhess == :c
        hessf = x -> hess_finite_diff(x, f, 5e-2)
    elseif FDhess == :Jfw
        hessf = x -> MyFiniteDiff.jacobian_finite_diff(
            (x -> MyFiniteDiff.grad_finite_diff(x, f, sqrt(eps()), :fw)),
            x,
            5e-2,
            :fw
        )
    elseif FDhess == :Jc
        hessf = x -> MyFiniteDiff.jacobian_finite_diff(
            (x -> MyFiniteDiff.grad_finite_diff(x, f, sqrt(eps()), :fw)),
            x,
            5e-2,
            :c
        )
    elseif FDhess == :MF
         # Matrix free, only defines matrix-hessian product
         error("Matrix-free method is unimplemented!")
    else
        error("Unknown method for Hessian finite differences")
    end

    
    k = 1
    xk = x0
    xseq = Vector{typeof(x0)}()
    btseq = Vector{Int}()
    gradfk = gradf(xk)
    push!(xseq, xk)
    
    while k < kmax
        if norm(gradfk) < tollgrad
            break
        end
        
        pk = hessf(xk) \ -gradfk
        
        # Backtracking
        alpha = alpha0
        bt = 1
        while f(xk + alpha * pk) > (f(xk) + c1 * alpha * gradf(xk)'*pk) && bt < btmax
            alpha = rho * alpha
            bt = bt + 1
        end
        push!(btseq, bt)
        
        xk = xk + alpha * pk
        k = k + 1
        gradfk = gradf(xk)
        push!(xseq, xk)
    end
    
    return xk, f(xk), gradfk, k, xseq, btseq
end
    
end

import Main.MyFiniteDiff

In [3]:
# pk could be obtained with an iterative method, for which only hessian-vector products are needed, thus avoiding building the whole hessian

# 1. Exercises

## 1.1 Gradient

In [4]:
@assert MyFiniteDiff.grad_finite_diff([3.], x -> x[1]^2, sqrt(eps()), :fw) ≈ [6]
@assert MyFiniteDiff.grad_finite_diff([3.], x -> x[1]^2, sqrt(eps()), :c) ≈ [6]
@assert MyFiniteDiff.grad_finite_diff([float(pi)], x -> sin(x[1]), sqrt(eps()), :fw) ≈ [-1]
@assert MyFiniteDiff.grad_finite_diff([float(pi)], x -> sin(x[1]), sqrt(eps()), :c) ≈ [-1]

## 1.2 Hessian

In [5]:
# Fails with sqrt(eps()). Big numerical cancellation problems
@assert isapprox(MyFiniteDiff.hess_finite_diff([3., 2.], x -> x[1]^2 + x[2]^2, sqrt(eps())*norm([3., 2.])), [2. 0.; 0. 2.], atol=1.2)
# much more precise with a bigger step
@assert isapprox(MyFiniteDiff.hess_finite_diff([3., 2.], x -> x[1]^2 + x[2]^2, 5e-2), [2. 0.; 0. 2.])

## 1.3 Jacobian

In [6]:
@assert MyFiniteDiff.jacobian_finite_diff(x -> [x[1]^2 + x[2]^2, 3*x[1] - 4*x[2]^2], [10, 10], sqrt(eps()), :fw) ≈ [20 20; 3 -80]
@assert MyFiniteDiff.jacobian_finite_diff(x -> [x[1]^2 + x[2]^2, 3*x[1] - 4*x[2]^2], [10, 10], sqrt(eps()), :c) ≈ [20 20; 3 -80]

### 1.3.1 Hessian as Jacobian of gradient

In [7]:
# Notice here how we have got even bigger numerical precision problems
@assert isapprox(MyFiniteDiff.jacobian_finite_diff(
    (x -> MyFiniteDiff.grad_finite_diff(x, x -> x[1]^2 + x[2]^2, sqrt(eps()), :fw)),
    [3., 2.],
    5e-2,
    :fw
), [2.0 0.0; 0.0 2.0], atol=1e-5)

## 1.4 Netwon Method with numerical gradient and Hessian

In [8]:
f1(x) = x[1]^2 + 4*x[2]^2 + 5

kmax = 1000
tollgrad = 1e-12
x0 = [1.0, 1.0]
alpha0 = 1.5
c1 = 1e-4
rho = 0.8
btmax = 50

xk, fk, gradfk, k, xseq, btseq = MyFiniteDiff.newton_method_backtrack_finite_diff(x0, f1, kmax, tollgrad, alpha0, c1, rho, btmax, :fw, :c)
@show xk
@show fk
@show gradfk
@show k

xk, fk, gradfk, k, xseq, btseq = MyFiniteDiff.newton_method_backtrack_finite_diff(x0, f1, kmax, tollgrad, alpha0, c1, rho, btmax, :fw, :Jfw)
@show xk
@show fk
@show gradfk
@show k

xk, fk, gradfk, k, xseq, btseq = MyFiniteDiff.newton_method_backtrack_finite_diff(x0, f1, kmax, tollgrad, alpha0, c1, rho, btmax, :fw, :Jc)
@show xk
@show fk
@show gradfk
@show k

xk = [-2.0861924161863305e-8, -1.1920972965678876e-8]
fk = 5.000000000000001
gradfk = [0.0, 0.0]
k = 28
xk = [-7.659289540317465e-9, -9.637928486669447e-9]
fk = 5.0
gradfk = [0.0, 0.0]
k = 29
xk = [-9.004309664913202e-9, -9.005858892366818e-9]
fk = 5.0
gradfk = [0.0, 0.0]
k = 28


28